# <center> **Progetto di Open Data Management 2022-2023**

<br>
<div align="center">
    <img src="op_wallpaper.jpg" width="1200" height="auto">

### <center> _Andrea Spinelli, Raffaele Terracino, Marco Valenti_
##### <center> 23 Giugno 

***
***

# __Indice__

###  [`1. Traccia`](#)
### [`2. Selezione dei dataset`](#)
##### &emsp;&emsp; [`2.1 Raccolta`](#)
##### &emsp;&emsp; [`2.2 Licenze`](#)
### [`3. Elaborazione dei dataset`](#)
##### &emsp;&emsp; [`3.1 Pulizia`](#)
##### &emsp;&emsp; [`3.2 Pivoting`](#)
##### &emsp;&emsp; [`3.3 Arricchimento`](#)
### [`4. Trasformazione dei dataset a 5 stelle`](#)
##### &emsp;&emsp; [`4.1 Ontologia`](#)
##### &emsp;&emsp; [`4.2 Creazione grafo RDF`](#)
### [`5. Data visualization`](#)
### [`6. Creazione di un'applicazione`](#)

***
***
# __1 Traccia__

<u> Utilizzando il linguaggio Python, per lo sviluppo del progetto si devono innanzitutto rispettare i seguenti passi: </u>

- _Selezione dati;_
- _Elaborazione dati (data cleaning, definizione struttura omogenea);_
- _Open Linked Data (creazione di uno strato semantico, ontologie, interlinking)._
 
<u> Dopodiché si può passare, opzionalmente, allo sviluppo del servizio tramite: </u> 
- _Creazione di un bot telegram._

<u> Siano i seguenti dataset (forniti in allegato): </u>

__farmacie.csv,__

__parafarmacie.csv,__

__strutture_sanitarie_pubbliche.csv,__

__strutture_sanitarie_private.csv.__

<u> Si vuole realizzare un servizio in ambito “Salute” che fornisca le posizioni e le disponibilità delle strutture mediche nella regione siciliana. </u>

Si suppone che un utente del servizio abbia bisogno di dover andare a fare una visita dermatologica (struttura privata), oppure una visita ad un consultorio (struttura pubblica), tuttavia non conosce la posizione della struttura; quindi, può interrogare il servizio affinché possa trovare in un raggio di 𝑥 metri, data la sua posizione, un elenco (messaggio testuale, posizioni, …) di strutture specializzate nel campo desiderato. 

È possibile inoltre che l’utente, dopo una visita, abbia la prescrizione di alcuni farmaci; pertanto, avrà bisogno della locazione delle farmacie, o parafarmacie, più vicine. Il servizio di tale richiesta funziona allo stesso modo di quello precedente, inoltre si può fornire anche il recapito telefonico per effettuare la prenotazione dei farmaci. 

***
***

***
***
# __2 Selezione dei dataset__


### __2.1 Raccolta__

***

### __2.2 Licenze__

***
***
# __3 Elaborazione dei dataset__

### __3.1 Pulizia__

***

### __3.2 Pivoting__

***

### __3.3 Arricchimento__

///some comments

In [ ]:
import pandas as pd
import re
comuni = pd.read_csv("../datasets/popolazione_sicilia.csv", sep=',', skiprows=[1])
farmacie = pd.read_csv('../datasets/synthesized_&_cleaned/farmacie.csv', sep=';')

comuni['Territorio'] = comuni['Territorio'].apply(lambda x : x.title().replace('à', 'a\'').replace('è', 'e\'').replace('ì', 'i\'').replace('ò', 'o\'').replace('ù', 'u\''))
farmacie['DESCRIZIONECOMUNE'] = farmacie['DESCRIZIONECOMUNE'].apply(lambda x : x.title())
joined = comuni.merge(farmacie, left_on='Territorio', right_on='DESCRIZIONECOMUNE')

df2=joined.drop_duplicates(subset=['Territorio'], keep='first')

df3=df2.rename(columns={"Territorio": "Comune", "Value": "PopolazioneTotale", "DESCRIZIONEPROVINCIA" : "Provincia", "SIGLAPROVINCIA":"SiglaProvincia"})
df3.to_csv('comuni_sicilia.csv', columns=['Comune', 'PopolazioneTotale', 'Provincia', 'SiglaProvincia'], index=False)

///some comments

In [ ]:
import pandas as pd

df_private = pd.read_csv("../datasets/synthesized_&_cleaned/strutture_sanitarie_private.csv", delimiter=";")
df_pubbliche = pd.read_csv("../datasets/synthesized_&_cleaned/strutture_sanitarie_pubbliche.csv", on_bad_lines="skip", delimiter=";", encoding="utf-8")
df_paraf = pd.read_csv("../datasets/cleaned/parafarmacie.csv", on_bad_lines="skip", delimiter=";", encoding="utf-8")
df_farma = pd.read_csv("../datasets/cleaned/farmacie.csv", on_bad_lines="skip", delimiter=";", encoding="utf-8")

df_private.columns = df_private.columns.str.strip()
df_pubbliche.columns = df_pubbliche.columns.str.strip()

///some comments

In [ ]:
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
from pathlib import Path
import re

geolocator = Nominatim(user_agent="stv_project")
geocode = RateLimiter(geolocator.geocode) #, min_delay_seconds=0.1

LOC = []
LAT = []
LON = []

for via, citta in zip(df_pubbliche["Indirizzo"], df_pubbliche["Citta\'"]):
    
    location = geocode(via + " " + citta)

    if(location is not None):
        cap = re.search(r'\d{5}',location.address)

        if cap is not None:
            LOC.append(cap.group())
        else:
            LOC.append(00000)

        if location.latitude is not None:
            LAT.append(location.latitude)
        else:
            LAT.append(0.0)

        if location.longitude is not None:
            LON.append(location.longitude)
        else:
            LON.append(0.0)
            
df_private.insert(loc=0, column='CAP', value=pd.Series(LOC))
df_private.insert(loc=0, column='LONGITUDINE', value=pd.Series(LON))
df_private.insert(loc=0, column='LATITUDINE', value=pd.Series(LAT))

filepath = Path('../datasets/private.csv')
df_private.to_csv(filepath, index=False)

///some comments

In [ ]:
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
from pathlib import Path
import re

geolocator = Nominatim(user_agent="stv_project")
geocode = RateLimiter(geolocator.geocode) #, min_delay_seconds=0.1

COM = []
LAT = []
LON = []

for via, cap in zip(df_pubbliche["Indirizzo"], df_pubbliche["CAP"]):

    location = geocode(via + " " + str(cap))

    if(location is not None):
        comune = re.search(r'(\w*), Sicilia', location.address)

        if comune is not None:
            COM.append(comune[1])
        else:
            COM.append('######')

        if location.latitude is not None:
            LAT.append(location.latitude)
        else:
            LAT.append(0.0)

        if location.longitude is not None:
            LON.append(location.longitude)
        else:
            LON.append(0.0)
            
df_pubbliche.insert(loc=0, column='COMUNE', value=pd.Series(COM))
df_pubbliche.insert(loc=0, column='LONGITUDINE', value=pd.Series(LON))
df_pubbliche.insert(loc=0, column='LATITUDINE', value=pd.Series(LAT))

filepath = Path('../datasets/pubbliche.csv')
df_pubbliche.to_csv(filepath, index=False)


***

***
***
# __4 Trasformazione dei dataset a 5 stelle__

### __4.1 Ontologia__

***

### __4.2 Creazione grafo RDF__

***
***
# __5 Data visualization__

***
***
# __6 Creazione di un'applicazione__
